# Supervised Learning - Classification Problem

## Students' dropout and academic success

### (Droupout, Enrolled, Graduate)

Faculdade: FEUP - Faculdade de Engenharia da Universidade do Porto

Curso: L.EIC - Licenciatura em Engenharia Informática e Computação 

Unidade Curricular: Inteligência Artificial

Ano Curricular: 2021/22

Grupo: 21_1D

Elementos:
- Henrique Ribeiro Nunes, up201906852@up.pt
- Margarida Assis Ferreira, up201905046@up.pt
- Patrícia do Carmo Nunes Oliveira, up201905427@up.pt

### Specification

O foco principal deste problema é analisar a informação conhecida sobre a matrícula do aluno (percurso acadêmico, demografia e fatores socioeconómicos) e o desempenho académico dos alunos no final do primeiro e segundo semestres. Com o objetivo de usar estes dados para construir modelos de classificação para **prever a desistência e o sucesso académico dos alunos**.

Este problema é um ***single label multiclass classification problem*** com 37 atributos:
- 36 métricas distintas para descrever as informações do aluno.
- 1 objetivo com 3 resultados possíveis (*Droupout*, *Enrolled*, *Graduate*).

Existe um **forte desbalanceamento** em relação a um dos resultados possíveis.


### Tools & Resources

* [Como balancear um dataset](https://towardsdatascience.com/how-to-balance-a-dataset-in-python-36dff9d12704)
* [Como lidar com desbalanceamento em ML](https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)
* [Método pandas.DataFrame.duplicated()](https://www.machinelearningplus.com/pandas/pandas-duplicated/)
* [Vizualização de distribuições de dados](https://seaborn.pydata.org/tutorial/distributions.html)
* [Guia de Scikit Learn](https://scikit-learn.org/stable/user_guide.html)
* [Plot de gráficos com múltiplas barras](https://www.geeksforgeeks.org/plotting-multiple-bar-charts-using-matplotlib-in-python/)
* [Grid Search para Decision Trees](https://ai.plainenglish.io/hyperparameter-tuning-of-decision-tree-classifier-using-gridsearchcv-2a6ebcaffeda)
* [Refinamento de hiperparâmetros com Grid Search](https://medium.datadriveninvestor.com/hyperparameter-tuning-with-deep-learning-grid-search-8630aa45b2da)

## Data Analysis

A análise dos dados é um passo importante nos problemas de classificação. 

Nesta secção são analisados os atributos que classificam os dados e o tipo e intervalo de valores de cada um dos atributos, bem como o tamanho do conjunto de dados, a presença de valores nulos ou amostras duplicadas. Igualmente é explorado a distribuição das classes e de valores por atributos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
SAVE_FILE = False

In [ ]:
# Load data
data = pd.read_csv("data/data_original.csv")
pd.set_option('max_columns', None)

In [ ]:
# Print head values and summary statistics
print(data.describe())
print()
print(data.head())
print()

In [ ]:
# Get all 36 features used and 3 possible results
features = list(data.columns)
features.remove("class")
print("Number of Features: {}".format(len(features)))
print("Features: {}".format(features))
values = list(data[features].values)

classes = list(data["class"].unique())
classes.sort()
print("\nNumber of Results: {}".format(len(classes)))
print("Result/Prediction: {}".format(classes))
targets = list(data["class"].values)

In [ ]:
# Check atribute types and values interval
for name, dtype in data.dtypes.iteritems():
    print("{} | {} | [{} , {}] ".format(name.ljust(46), str(dtype).ljust(7), data[name].min(), data[name].max()))

In [ ]:
# Data size
print("Data Size: {}".format(len(data)))

In [ ]:
# Check if there are columns with N/A values
print("N/A values found: {}".format(data.isnull().values.any()))

In [ ]:
# Check if there are duplicated Data
bool_series = data.duplicated()
print(bool_series)

old_size = len(data)

# Removing all duplicated data if exists
data = data[~bool_series] 

new_size = len(data)

# check if there were actualy duplicated data
print()
if (new_size == old_size):
    print("No data was removed: there were no duplicated data")
else:
    print("Was found and removed {} duplicated data".format(old_size-new_size))

In [ ]:
# Resultant data without duplicated entries
data

In [ ]:
# Count each class
def countEachClass(data):
    n_dropouts = data['class'].value_counts().Dropout
    n_enrolled = data['class'].value_counts().Enrolled
    n_graduate = data['class'].value_counts().Graduate
    return n_dropouts, n_enrolled, n_graduate

In [ ]:
# Get data rows associated with each class
def sampleEachClass(data):
    class_dropout = data[data['class'] == "Dropout"]
    class_enrolled = data[data['class'] == "Enrolled"]
    class_graduate = data[data['class'] == "Graduate"]
    return class_dropout, class_enrolled, class_graduate

In [ ]:
# Class Distribution
# Check data balance
n_dropouts, n_enrolled,n_graduate = countEachClass(data)
print("Number of 'Dropout' occurences: {}".format(n_dropouts))
print("Number of 'Enrolled' occurences: {}".format(n_enrolled))
print("Number of 'Graduate' occurences: {}".format(n_graduate))
print()

# Corresponding plot 
count_result = pd.DataFrame(data["class"]).value_counts().rename_axis("class").reset_index(name="count")
print(count_result)

# Bar plot
plot_count_res = sb.barplot(data=count_result, x="count", y="class")
plt.show()

# Pie plot with percentages
plt.pie([n_graduate, n_dropouts, n_enrolled], autopct = '%0.00f%%')
plt.show()

In [ ]:
# Histogram for each feture values and respective count of classses

def count_targets_for_each_value(data, feat):
    data_aux = data[[feat, "class"]]
    d = {}
    for line in data_aux.values:
        if not line[0] in d:
            d[line[0]] = {classes[0]: 0, classes[1]: 0, classes[2]: 0}
        d[line[0]][line[1]] += 1

    for entry in d.keys():
        d[entry] = [d[entry][c] for c in classes]
    aux = pd.DataFrame(d, index=classes)
    return aux.reindex(sorted(aux.columns), axis=1)


for i in range(36):
    plt.title(features[i])
    plt.hist(data[features[i]].values)
    plt.show()
    print(count_targets_for_each_value(data, features[i]))

Analisando a distribuição de cada atributo pelas classes possíveis, podemos concluir que existem alguns atributos irrelevantes, na medida que não se distingue à priori nem se obtem qualquer informação sobre qual a classe mais provável para uma nova amostra com base no valor desse atributo, como conseguimos identifificar no caso do atributo 'Curricular units 2nd sem (credited)'

In [ ]:
# Distribution of each attribute 
def densityPlot(x, hue, fill):
    sb.displot(data, x=x, hue=hue, kind="kde", fill=fill)
    
# all 36 distinct attributes
attributes = list(data.columns)
attributes.remove("class")
    
for attribute in attributes:
    densityPlot(attribute, 'class', True)

In [ ]:
# Correlation Heatmap

corr = data.corr()

sb.set(rc = {'figure.figsize': (15,15) })
ax = sb.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sb.diverging_palette(20, 240, n=200),
    square=True,
    xticklabels=True,
    yticklabels=True,
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);
plt.show()

sb.reset_orig()

**Propriedades do Problema:** (a partir da análise dos dados de entrada)

- Nominal and Discrete attributes (including some binary ones)
- Dimensionality = 37 attibutos
- Size = 4424
- Type = Data Matrix
- No missing or duplicate Data
- No meaningful outliers
- Imbalance data

## Data Preprocessing

Tendo em conta as conclusões obtidas pela a análise dos dados originais mostrada anteriormente, podemos apercebermo-nos que é necessário realizar um pré-processamento dos dados, com o objetivo de resolver o maior problema adjacente a estes: a falta de balanceamento entre as diferentes classes de alvo. 

Para tal podemos usar entre outras estratégias uma das seguintes:
- **oversampling** : «Aumentar o número de amostras/entradas da menor classe até coincidir com o tamanho da maior classe»
- **undersampling** : «Diminuir o número de amostras/entradas da maior classe até coincidir com o tamanho da menor classe»

Em qualquer uma das abordagens acima, a escolha das amostras escolhidas para serem retidas ou replicadas com pequenas modificações é aleatória.

Foi também necessário proceder à padronização dos dados para tornar a existência de _oultiers_ menos relevante e equiparar a escala de cada atributo para assim melhorar a performance de alguns algoritmos.

### Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

def standardize(data, to_standardize):
    data_to_standardize = data[to_standardize]
    scaler = StandardScaler()
    stand_values = scaler.fit_transform(data_to_standardize.values)

    stand_values_df = pd.DataFrame(
        stand_values, 
        index=data_to_standardize.index, 
        columns=to_standardize)
    data[to_standardize] = stand_values_df[to_standardize]
    return data

In [ ]:
non_binary_features = [feature for feature in features if len(data[feature].unique()) != 2]
data_standard = standardize(data.copy(), non_binary_features)

### Undersampling

Uma das técnicas para lidar com o desbalanceamento de classes em machine lerning é chamado de *undersampling*. Esta técnica de balanceamento consiste em remover algumas observações das classes majoritárias, até que as classes majoritárias e minoritárias sejam equilibradas. A técnica *undersampling* pode ser uma boa escolha quando temos dados desequilibrados, mas uma desvantagem é que removemos informações que podem ser valiosas.

Para remover as observações das classes majoritárias, usamos a função `sample(sequence, k)`, uma função do módulo `Random` de Python, que retorna uma lista de comprimento `k` de itens escolhidos aleatoriamente de `sequence`.

In [ ]:
# Imbalance Original Data
print("Classes count:")
print(data['class'].value_counts())

data['class'].value_counts().plot(kind='bar', title='count (target)')

In [ ]:
def undersampling(data):
    n_dropouts,n_enrolled,n_graduate = countEachClass(data)
    class_dropout,class_enrolled,class_graduate = sampleEachClass(data)
    
    class_dropout_under = class_dropout.sample(n_enrolled, replace=True)
    class_graduate_under = class_graduate.sample(n_enrolled, replace=True)
    return pd.concat([class_dropout_under, class_graduate_under, class_enrolled], axis=0)

Agora temos os nossos dados balanceados, como é possível observer no gráfico criado pelo código abaixo.

In [ ]:
data_under = undersampling(data)

# plot the count after under-sampeling
print("Classes count after under-sampling:")
print(data_under['class'].value_counts())

data_under['class'].value_counts().plot(kind='bar', title='count (target)')

In [ ]:
# Save to file
if SAVE_FILE: data_under.to_csv("data/data_under.csv", index=False)

### Oversampling

As entradas da classe menor sáo replicadas até totalizarem o número de amostras da classe maior.

In [ ]:
n_dropouts,n_enrolled,n_graduate = countEachClass(data)

print("DROPOUT: {} | ENROLLED: {} | GRADUATE: {}".format(n_dropouts, n_enrolled, n_graduate))

In [ ]:
# Imbalance Original Data
unbalanced_count = data['class'].value_counts()
unbalanced_count.plot.bar()
plt.show()

#### Random Over-Sampling

«Oversampling can be defined as adding more copies to the minority class.»

**Desvantagens:** pode causar *overfitting* e pobre generalização do conjunto de dados para teste.

In [ ]:
def oversampling(data):
    n_dropouts,n_enrolled,n_graduate = countEachClass(data)
    dropout_samples,enrolled_samples,graduate_samples = sampleEachClass(data)

    dropout_samples_over = dropout_samples.sample(n_graduate, replace=True)
    enrolled_samples_over = enrolled_samples.sample(n_graduate, replace=True)

    return pd.concat([graduate_samples, dropout_samples_over, enrolled_samples_over], axis=0)

In [ ]:
data_over = oversampling(data)
print("Total dintinct classes: \n{}".format(data_over['class'].value_counts()))

rnd_oversampling_count = data_over['class'].value_counts()
rnd_oversampling_count.plot.bar()
plt.show()

In [ ]:
# Save to file
if SAVE_FILE: data_over.to_csv("data/data_over.csv", index=False)

É importante referir que nenhuma das soluções acima é uma solução perfeita, pois a aplicação de undersampling pode implicar a perda de infromação, da mesma forma que a aplicação de oversampling (sem qualquer modificação das amostras escolhidas aleatóriamente para serem replicadas) pode levar a um posterior overfitting dos modelos gerados a estes novos dados.

### Combine under and over sampling

Tendo isto em conta a seguinte tentativa tenta encontrar um meio termos entre as soluções anteriores, fazendo as classes em questão convergir para um valor mediano e não para um máximo nem minímo, tentando combater as consequencias sentidas nos dados ao aplicar isoladamente cada uma das estratégias, obtando por alcaçar um meio termo.

In [ ]:
# make the counts meet at the middle point
# in this case the middle point is consider to be the dropout class
def combine_sampling(data):
    n_dropouts,n_enrolled,n_graduate = countEachClass(data)
    dropout_samples,enrolled_samples,graduate_samples = sampleEachClass(data)
    graduate_samples_combine = graduate_samples.sample(n_dropouts, replace=True)
    enrolled_samples_combine = enrolled_samples.sample(n_dropouts, replace=True)
    return pd.concat([graduate_samples_combine, dropout_samples, enrolled_samples_combine], axis=0)

In [ ]:
data_combine = combine_sampling(data)

print("Total dintinct classes: \n{}".format(data_combine['class'].value_counts()))

combine_count = data_combine['class'].value_counts()
combine_count.plot.bar()
plt.show()

In [ ]:
# Save to file
if SAVE_FILE: data_combine.to_csv("data/data_combine.csv", i)

## Learning Algorithms

### Identification of the Target Concept

A pergunta a que queremos responder é a seguinte: "Tendo em conta o percurso académico de um aluno e outros fatores externos como é que conseguimos prever se este vai desistir, graduar ou continuar no mesmo ano?"

De forma a avaliar os modelos posterioemente criados podemos escolher nesta secção se os dados a usar ai foram proviamente standarizados e qual o tipo de estratégia para corrigir o balanceamento que é utilizada (inclusive nenhuma). 

In [ ]:
# Parameteres of the pre-processing and construction of the model for the data
STANDARDIZATION = True
BALANCETECNIQUE = 'OVER'

# original data
data = pd.read_csv('data/data_original.csv')

if STANDARDIZATION:
    non_binary_features = [feature for feature in features if len(data[feature].unique()) != 2]
    data = standardize(data, non_binary_features)

if BALANCETECNIQUE == 'UNDER': data = undersampling(data)
elif BALANCETECNIQUE == 'OVER': data = oversampling(data)
elif BALANCETECNIQUE == 'COMBINE': data = combine_sampling(data)

values = list(data[list(data.columns[:-1])].values)
targets = list(data['class'].values)

GRIDSEARCH = False

### Validation

#### Train test split

In [ ]:
from sklearn.model_selection import train_test_split
def split_data(data, classes, test_size):
  feat_train, feat_test, target_train, target_test = train_test_split(data, classes, test_size=test_size, shuffle=True)
  return feat_train, feat_test, target_train, target_test

#### Cross validation

In [ ]:
from sklearn.model_selection import cross_val_score
def cross_validation(model, features, targets, cv):
    scores = cross_val_score(model, features, targets, cv=cv)
    return scores

#### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
def confusion_matrix(target_test, target_predictions):    
    return metrics.confusion_matrix(target_test, target_predictions)

def display_confusion_matrix(cm):
    metrics.ConfusionMatrixDisplay(cm, display_labels=classes).plot()

#### Validate model

In [ ]:
from sklearn import metrics

def recall(cm, i):
    return cm[i][i]/sum(cm[i])

def precision(cm, i):
    cmt = np.copy(cm).transpose()
    return cmt[i][i]/sum(cmt[i])

def f_measure(cm, i):
    p = precision(cm, i)*100
    r = recall(cm, i)*100
    return 2 * (p * r) / (p + r)

def validate(model, features, targets):
    test_size = 0.2
    cross_validation_split = 5
    feat_train, feat_test, target_train, target_test = split_data(features, targets, test_size)
    
    model.fit(feat_train, target_train)
    predictions = model.predict(feat_test)
    
    acc = metrics.accuracy_score(target_test, predictions)
    print("Accuracy: {:.2f}\n".format(acc*100))
    
    cm = confusion_matrix(target_test, predictions)
    print("Precision:")
    for i in range(3): print("\t{:.2f}% - {}".format(precision(cm, i)*100, classes[i]))
    print("Recall:")
    for i in range(3): print("\t{:.2f}% - {}".format(recall(cm, i)*100, classes[i]))
    print("F1-measure:")
    for i in range(3): print("\t{:.2f}% - {}".format(f_measure(cm, i), classes[i]))
        
    scores = cross_validation(model, features, targets, cross_validation_split)
    print("Cross validation {}-fold: {}".format(cross_validation_split, [round(x, 3) for x in scores]))
    print("\t{:.2f} - accuracy".format(scores.mean()))
    print("\t{:.2f} - standard deviation".format(scores.std()))
    
    test_report = metrics.classification_report(target_test, predictions)
    print("\nTest Classification Report:\n", test_report)
    
    predictions_train = model.predict(feat_train)
    train_report = metrics.classification_report(target_train, predictions_train)
    print("\nTrain Classification Report:\n", train_report)

    display_confusion_matrix(cm)
    
    return test_report, train_report

In [ ]:
from sklearn.model_selection import GridSearchCV

def grid_search(model, grid_params, features, targets):
    test_size = 0.2
    cross_validation_split = 5
    feat_train, feat_test, target_train, target_test = split_data(features, targets, test_size)
    
    gs = GridSearchCV(
        model,
        grid_params,
        verbose = 1,
        n_jobs = 1,
        cv = cross_validation_split,
    )
    
    gs_results = gs.fit(feat_train, target_train)
    
    print("best score: " + str(gs_results.best_score_))
    print("best estimator: " + str(gs_results.best_estimator_))
    print("best parameters: " + str(gs_results.best_params_))
    
    return gs_results.best_estimator_

### Algorithms

#### Decision Tree Classifier

In [ ]:
PLOT_TREE=False
DOWNLOAD_TREE=False

In [ ]:
grid_params_dtc = {
    'criterion': ['gini', 'entropy', 'log_loss'], 
    'splitter': ['best', 'random'],
    'max_depth': [5, 7, 10, 20],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None, 5, 10, 20],
}

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(
  criterion= 'entropy', 
  splitter= 'best', 
  max_depth= 7, 
  max_features = None, 
  max_leaf_nodes= None) 

if GRIDSEARCH: dtc = grid_search(DecisionTreeClassifier(), grid_params_dtc, values, targets)

test_report_dtc, train_report_dtc = validate(dtc, values, targets)

In [ ]:
# Plot tree
if PLOT_TREE: 
    from sklearn.tree import plot_tree
    plot_tree(dtc)

In [ ]:
# Create PDF with the tree
if DOWNLOAD_TREE:
    import graphviz
    from sklearn.tree import export_graphviz

    dot_data = export_graphviz(dtc, out_file=None, feature_names=features, class_names=targets ) 
    graph = graphviz.Source(dot_data) 
    graph.render("decision_tree") 


#### K-nearest Neighbour

In [ ]:
grid_params_knn = {
    'n_neighbors': [3,5,11,19], 
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan'],
}

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(
    n_neighbors = 9,
    weights = 'distance',
    metric = 'manhattan')

if GRIDSEARCH: knn = grid_search(KNeighborsClassifier(), grid_params_knn, values, targets)

test_report_knn, train_report_knn = validate(knn, values, targets)

#### Neural Network

In [ ]:
grid_params_nnc = {
    'solver': ['lbfgs', 'sgd', 'adam'], 
    'alpha': [1e-5, 1e-2],
    'hidden_layer_sizes': [(5, 2), (40,50)],
    'max_iter':[5000, 200],
    'random_state':[None, 1, 36]
}

In [ ]:
from sklearn.neural_network import MLPClassifier

nnc = MLPClassifier(
    solver = 'sgd', 
    alpha = 1e-5,
    hidden_layer_sizes = (40, 50),
    max_iter = 5000,
    random_state = 1,
    activation = 'relu',
    learning_rate = 'constant',
    learning_rate_init = 0.01)

if GRIDSEARCH and False: nnc = grid_search(MLPClassifier(), grid_params_nnc, values, targets)

test_report_nnc, train_report_nnc = validate(nnc, values, targets)

#### Support Vector Machine

In [ ]:
grid_params_svm = {
    'C': [0.1, 1, 10, 100], 
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'decision_function_shape': ['ovr', 'ovo']
}

In [ ]:
from sklearn.svm import SVC

svm = SVC(
    decision_function_shape='ovr',
    C = 10,
    gamma=0.01,
    kernel='rbf')

if GRIDSEARCH: svm = grid_search(SVC(), grid_params_svm, values, targets)

test_report_svm, train_report_svm = validate(svm, values, targets)

#### Random Forest

In [ ]:
grid_params_rfc = {
    'max_depth': [None, 10, 50],
    'max_features': ['sqrt', 'log2', 15, 25],
    'min_samples_leaf': [1, 3, 4],
    'min_samples_split': [2, 10],
    'n_estimators': [20, 40]
}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(
    n_estimators=40,
    max_depth=None,
    max_features=None,
    min_samples_leaf=1,
    min_samples_split=2)

if GRIDSEARCH: rfc = grid_search(RandomForestClassifier(), grid_params_rfc, values, targets)

test_report_rfc, train_report_rfc = validate(rfc, values, targets)

#### Extra Trees

In [ ]:
grid_params_etc = {
    'max_depth': [None, 10, 50, 100],
    'random_state': [None, 5],
    'min_samples_split': [2, 10],
    'n_estimators': [20, 40, 60]
}

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

etc = ExtraTreesClassifier(
    n_estimators=40, 
    max_depth=100,
    min_samples_split=2, 
    random_state=None)

if GRIDSEARCH: etc = grid_search(ExtraTreesClassifier(), grid_params_etc, values, targets)
    
test_report_etc, train_report_etc = validate(etc, values, targets)

### Evaluation of the learning process

No que diz respeito à avaliação do processo de aprendizagem foi implementada a função `validate` (apresentada na secção de *Validation*) que é responsável por dividir os dados em dados de treino e de teste, bem como avaliar os resultados obtidos em cada um dos modelos criados usando as métricas apropriadas: ***accuracy***, ***precision***, ***recall***, ***F-measure*** e ***confusion matrix***, tendo sempre em consideração as características do problema: o problema trata-se um um problema de classificação ***multiclass single label*** com 3 diferentes classes objetivo (3 *targets*). Esta avaliação centra-se principalmente no conjunto de dados de teste.

### Results Comparision

O resultados apresentados foram obtidos apartir da comparação da avaliação de performance dos diferentes modelos considerando as seguintes combinações nas variações da data de entrada:

- aplicação de **standarização** aos dados: *yes/no*
- estratégia para **tratamento do desbalanceamento** dos dados: *none/under/over/combined*

NOTA: Estes parametros podem ser ajustados e configurados no início da secção ***Learning Algorithms, «Identification of the Target Concept»***. Os parametros de cada um dos algoritmos resultaram de uma pesquisa ***Grid Search*** efetuada previamente, que por razões de tempo e performance por defeito é desabilitada.

Neste sentido foram comparados os resultados obtidos para cada uma das combinações possíveis para os 3 algoritmos apresentados anteriormente : ***Decision Tree Classifier***, ***K-Nearest Neighbor***, ***Neural Networks***, ***Support Vector Machine***, ***Random Forest***, ***Extra Trees***, resultando nos seguintes gráficos, baseados nas tabelas retornadas pela função de `validate` apresentados em seguida.

In [ ]:
print("-- CONTEXT:: STANDARDIZATION: {} | BALANCETECNIQUE: {} | GRIDSEARCH: {} -- ".format(STANDARDIZATION, BALANCETECNIQUE, GRIDSEARCH))
print()

def get_x_groups(report):
    X_GROUPS = []
    
    lines = list(filter(lambda line: len(line) > 1, report.split('\n')))
    for line in lines[1:]:
        cells = list(filter(lambda cell: cell != '', line.split(' ')))
        if len(cells) <= 5: X_GROUPS.append(cells[0])
        else: X_GROUPS.append(cells[0] + ' ' + cells[1])
    return X_GROUPS

def get_fm_values(report):
    f_values = []

    lines = list(filter(lambda line: len(line) > 1, report.split('\n')))
    for line in lines[1:]:
        cells = list(filter(lambda cell: cell != '', line.split(' ')))
        f_values.append(int(float(cells[-2])*100))
    return f_values

def comparision_plot(set_label, X, v_dtc, v_knn, v_nnc, v_svm, v_rfc, v_etc):
    plt.rcParams["figure.figsize"] = (8, 5)
    
    X_axis = np.arange(len(X))
    width = 0.14
    
    plt.bar(X_axis-width*2.5, v_dtc, width, label = 'DTC', color='sandybrown')
    plt.bar(X_axis-width*1.5, v_knn, width, label = 'KNN', color='lightgreen')
    plt.bar(X_axis-width*0.5, v_nnc, width, label = 'NNC', color='lightskyblue')
    plt.bar(X_axis+width*0.5, v_svm, width, label = 'SVM', color='lightcoral')
    plt.bar(X_axis+width*1.5, v_rfc, width, label = 'RFC', color='gold')
    plt.bar(X_axis+width*2.5, v_etc, width, label = 'ETC', color='plum')
    
    plt.xticks(X_axis, X)
    plt.xlabel("Metrics")
    plt.ylabel("Assert rate")
    plt.title("F1 measure performance comparision in {} set \n (Standardization: {} | Balance: {})".format(set_label, STANDARDIZATION, BALANCETECNIQUE))
    plt.legend(loc='lower right')
    plt.show()
    
def reports_comparision(set_label, dtc, knn, nnc, svm, rfc, etc):
    X_report = get_x_groups(dtc)
    
    v_dtc = get_fm_values(dtc)
    v_knn = get_fm_values(knn)
    v_nnc = get_fm_values(nnc)
    v_svm = get_fm_values(svm)
    v_rfc = get_fm_values(rfc)
    v_etc = get_fm_values(etc)
    
    comparision_plot(set_label, X_report[:-2], v_dtc[:-2], v_knn[:-2], v_nnc[:-2], v_svm[:-2], v_rfc[:-2], v_etc[:-2])
    
# test set performance comparision
reports_comparision('test', test_report_dtc, test_report_knn, test_report_nnc, test_report_svm, test_report_rfc, test_report_etc)

# train set performance comparision
reports_comparision('train', train_report_dtc, train_report_knn, train_report_nnc, train_report_svm, train_report_rfc, train_report_etc)

### Receiver Operating Characteristic (ROC) Curve

A _ROC curve_ é uma representação gráfica que ilustra o desempenho de um sistema de classificação binário à medida que o seu _threshold_ varia.

Como o nosso problema é um _single label multiclass classification_ com três classes possíveis, calculamos **3 _ROC curves_**:
* *Enrolled* vs *Não Enrolled*
* *Graduate* vs *Não Graduate*
* *Dropout* vs *Não Dropout*

NOTA: O algoritmo usado para para o cálculo da ROC curve pode ser alterado. Para tal, basta alterar as variáveis `model` e `model_name` conforme o algoritmo desejado.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
import itertools as it

def roc(X, y, classes, model, model_name):
    # Binarize the output
    y = label_binarize(y, classes=classes)
    n_classes = y.shape[1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    classifier = OneVsRestClassifier(model)
    y_score = None
    if model_name == "SVM": y_score = classifier.fit(X_train, y_train).decision_function(X_test)
    else: y_score = classifier.fit(X_train, y_train).predict_proba(X_test)

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    colors = it.cycle(['blue', 'red', 'green'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(classes[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC for {}'.format(model_name))
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Plot roc curves for different models
# Change variables according to what model you want to test
# NOTE: Use model_name="SVM" when testing SVC models

model = dtc
model_name = "Decision Tree"

roc(data[features], data['class'], classes, model, model_name)

### Overfit - Validation Curve

É possível detetar o _overfit_ dos modelos se o _score_ obtido em dados de treino for significativamente maior do que o _score_ obtido em dados de teste. Para isso utilizámos uma curva de validação para comparar os dois _scores_, para a qual é necessário fazer variar um determinado parâmetro do modelo para comparar os dois conjuntos de dados ao longo do eixo das abcissas.

In [ ]:
from sklearn.model_selection import validation_curve

def overfit(model, model_name, values, targets, param_name, param_range):
    train_scores, test_scores = validation_curve(
        model,
        values,
        targets,
        param_name=param_name,
        param_range=param_range,
        scoring="accuracy",
        n_jobs=4,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.title("Validation Curve - {} - {}".format(model_name, param_name))
    plt.xlabel(param_name)
    plt.ylabel("Score")
    plt.ylim(0.5, 1.1)
    lw = 2
    plt.plot(param_range, train_scores_mean, label="Training score", color="darkorange", lw=lw)
    plt.fill_between(
        param_range,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.2,
        color="darkorange",
        lw=lw,
    )
    plt.plot(param_range, test_scores_mean, label="Cross-validation score", color="navy", lw=lw)
    plt.fill_between(
        param_range,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.2,
        color="navy",
        lw=lw,
    )
    plt.legend(loc="best")
    plt.show()

In [ ]:
# Change arguments of overfit call according to what you want to test

param_range = list(range(0, 51, 5))

overfit(nnc, "Neural Network", values, targets, 'hidden_layer_sizes', param_range)